# Q1

参考数据中给出了拱墅区街道划分（示意）`gongshu.geojson`和拱墅区POI数据（示意）`gs_poi.geojson`,

请根据这两个文件，按以下指标对拱墅区各街道的社会宜居性做初略的评价：

- 辖区内小学密度：
    - 辖区内小学的POI数据KIND6属性为`'160102'`


- 综合医院()的覆盖密度：
    - 认为综合医院的覆盖范围为1600米
    - 综合医院的POI数据KIND6属性为`'170101'`

将以上两个指标归一化后直接相加作为社会宜居性得分，并按照得分降序排列给出结果

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
gongshu_street = gpd.read_file('data/gongshu.geojson')
gongshu_poi = gpd.read_file('data/gs_poi.geojson')
gongshu_hospital = gongshu_poi.loc[gongshu_poi['KIND6'] == '170101']
gongshu_hospital.geometry = gongshu_hospital.to_crs(crs='EPSG:32651').buffer(1600)

In [ ]:
gongshu_hospital = gongshu_hospital.overlay(gongshu_street.to_crs(crs='EPSG:32651'), how='identity')
gongshu_hospital['area'] = gongshu_hospital.area
gongshu_school = gongshu_poi.loc[gongshu_poi['KIND6']  == '160102'].overlay(gongshu_street, how='identity')

In [ ]:
df = pd.DataFrame(columns=['Name', 'Hospital', 'School'])
for i in range(len(gongshu_street.index)):
    street_name = gongshu_street.iloc[i]['政区名称']
    street_hospital = gongshu_hospital.loc[gongshu_hospital['政区名称'] == street_name]
    street_school = gongshu_school.loc[gongshu_school['政区名称'] == street_name]
    df.loc[i] = [street_name, street_hospital['area'].sum(), len(street_school.index)]
df

In [ ]:
Livability = []
max_school, min_school = df['School'].max(), df['School'].min()
max_hospital, min_hospital = df['Hospital'].max(), df['Hospital'].min()
for i in range(len(df.index)):
    df.loc[i, 'Hospital'] = (df.iloc[i].Hospital - min_hospital) / (max_hospital - min_hospital)
    df.loc[i, 'School'] = (df.iloc[i].School - min_school) / (max_school - min_school)
    Livability.append(df.iloc[i].Hospital + df.iloc[i].School)
df['Livability'] = Livability
df.sort_values(by=['Livability'], ascending=False)